In [1]:
import torch

In [2]:
%%latex
$$C_{ij}=C(o_{ij})=-\bar{P_{ij}}log(P_{ij})-(1-\bar{P_{ij}})log(1-P_{ij})$$

<IPython.core.display.Latex object>

In [3]:
%%latex
$$o_{ij}=f(x_i)-f(x_j)$$

<IPython.core.display.Latex object>

In [4]:
%%latex
$$P_{ij}=\frac{e^{o_{ij}}}{1+e^{o_{ij}}}$$

<IPython.core.display.Latex object>

In [5]:
# torch.sigmoid()

In [6]:
%%latex
$$\text{out}_{i} = \frac{1}{1 + e^{-\text{input}_{i}}}$$

<IPython.core.display.Latex object>

In [7]:
class RankNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1)
        )
        
        self.out_activation = torch.nn.Sigmoid()
        
    def forward(self, input_1, input_2):
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)
        
        logits_diff = logits_1 - logits_2
        out = self.out_activation(logits_diff)
        
        return out
    
    def predict(self, inp):
        logits = self.model(inp)
        return logits

In [8]:
ranknet_model = RankNet(num_input_features=10)

In [9]:
inp_1, inp_2 = torch.rand(4, 10), torch.rand(4, 10)
# batch_size x input_dim
inp_1

tensor([[0.7739, 0.2678, 0.7803, 0.2465, 0.1128, 0.5217, 0.3234, 0.6663, 0.4072,
         0.3517],
        [0.9114, 0.3861, 0.2998, 0.7287, 0.0845, 0.2013, 0.8940, 0.7416, 0.4034,
         0.5031],
        [0.2257, 0.5093, 0.0579, 0.8604, 0.9059, 0.8036, 0.6285, 0.2813, 0.1195,
         0.4764],
        [0.7247, 0.6890, 0.5470, 0.7227, 0.3084, 0.2587, 0.5034, 0.9606, 0.8945,
         0.2966]])

In [10]:
preds = ranknet_model(inp_1, inp_2)
preds

tensor([[0.4603],
        [0.4677],
        [0.5009],
        [0.4907]], grad_fn=<SigmoidBackward0>)

In [11]:
first_linear_layer = ranknet_model.model[0]

In [12]:
first_linear_layer.weight.grad

In [13]:
criterion = torch.nn.BCELoss()
loss = criterion(preds, torch.ones_like(preds))
loss.backward()

In [14]:
first_linear_layer.weight.grad

tensor([[-0.0866, -0.0495, -0.0639, -0.0362,  0.0156, -0.0378, -0.0627, -0.0879,
         -0.0283, -0.0124],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0361, -0.0123,  0.0030,  0.0143, -0.0234, -0.0231,  0.0449,  0.0545,
         -0.0221, -0.0455],
        [ 0.0036, -0.0424, -0.0094, -0.0144, -0.0263, -0.0590, -0.0046,  0.0239,
         -0.0057, -0.0516],
        [ 0.0406,  0.0066,  0.0083,  0.0174, -0.0080,  0.0058,  0.0281,  0.0514,
          0.0030, -0.0077],
        [ 0.0383,  0.0211,  0.0202,  0.0227,  0.0006,  0.0197,  0.0328,  0.0411,
          0.0141,  0.0012],
        [ 0.0069, -0.0023,  0.0006,  0.0027, -0.0044, -0.0044,  0.0085,  0.0103,
         -0.0042, -0.0086],
        [-0.0397,  0.0136, -0.0033, -0.0158,  0.0257,  0.0254, -0.0493, -0.0599,
          0.0243,  0.0500],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.0015, -

In [15]:
ranknet_model.zero_grad()

# ListNet

In [16]:
from itertools import combinations
import numpy as np

from utils import ndcg, num_swapped_pairs

In [17]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )
        
    def forward(self, input_1):
        logits = self.model(input_1)
        return logits

In [18]:
%%latex
$$CE = -\sum ^{N}_{j=1} (P_y^i(j) * log(P_z^i(j)))$$

<IPython.core.display.Latex object>

In [19]:
%%latex
$$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$$

<IPython.core.display.Latex object>

In [20]:
def listnet_ce_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i))

def listnet_kl_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i/P_y_i))

def make_dataset(N_train, N_valid, vector_dim):
    fake_weights = torch.randn(vector_dim, 1)
    
    X_train = torch.randn(N_train, vector_dim)
    X_valid = torch.randn(N_valid, vector_dim)
    
    ys_train_score = torch.mm(X_train, fake_weights)
    ys_train_score += torch.randn_like(ys_train_score)
    
    ys_valid_score = torch.mm(X_valid, fake_weights)
    ys_valid_score += torch.randn_like(ys_valid_score)
    
#     bins = [-1, 1] # 3 relevances
    bins = [-1, 0, 1, 2] # 5 relevances
    
    ys_train_rel = torch.Tensor(
        np.digitize(ys_train_score.clone().detach().numpy(), bins=bins)
    )
    ys_valid_rel = torch.Tensor(
        np.digitize(ys_valid_score.clone().detach().numpy(), bins=bins)
    )
    
    return X_train, X_valid, ys_train_rel, ys_valid_rel

In [21]:
N_train = 1000
N_valid = 500

vector_dim = 100
epochs = 2

batch_size = 16

X_train, X_valid, ys_train, ys_valid = make_dataset(N_train, N_valid, vector_dim)

net = ListNet(num_input_features=vector_dim)
opt = torch.optim.Adam(net.parameters())

In [22]:
torch.unique(ys_train)

tensor([0., 1., 2., 3., 4.])

In [24]:
for epoch in range(epochs):
    idx = torch.randperm(N_train)
    
    X_train = X_train[idx]
    ys_train = ys_train[idx]
    
    cur_batch = 0
    for it in range(N_train // batch_size):
        batch_X = X_train[cur_batch: cur_batch + batch_size]
        batch_ys = ys_train[cur_batch: cur_batch + batch_size]
        cur_batch += batch_size
        
        
        opt.zero_grad()
        if len(batch_X) > 0:
            batch_pred = net(batch_X)
            # batch_loss = listnet_kl_loss(batch_ys, batch_pred)
            batch_loss = listnet_ce_loss(batch_ys, batch_pred)
            batch_loss.backward(retain_graph=True)
            opt.step()
            
        if it % 10 == 0:
            with torch.no_grad():
                valid_pred = net(X_valid)
                valid_swapped_pairs = num_swapped_pairs(ys_valid, valid_pred)
                ndcg_score = ndcg(ys_valid, valid_pred)
            print(f"epoch: {epoch + 1}.\tNumber of swapped pairs: "
                  f"{valid_swapped_pairs} / {N_valid * (N_valid - 1) // 2}\t"
                  f"nDCG: {ndcg_score:.4f}"
                 )
            
    

epoch: 1.	Number of swapped pairs: 35029 / 124750	nDCG: 0.8773
epoch: 1.	Number of swapped pairs: 32662 / 124750	nDCG: 0.8928
epoch: 1.	Number of swapped pairs: 30566 / 124750	nDCG: 0.9006
epoch: 1.	Number of swapped pairs: 28502 / 124750	nDCG: 0.9086
epoch: 1.	Number of swapped pairs: 26401 / 124750	nDCG: 0.9149
epoch: 1.	Number of swapped pairs: 24405 / 124750	nDCG: 0.9217
epoch: 1.	Number of swapped pairs: 22376 / 124750	nDCG: 0.9316
epoch: 2.	Number of swapped pairs: 21930 / 124750	nDCG: 0.9332
epoch: 2.	Number of swapped pairs: 20245 / 124750	nDCG: 0.9410
epoch: 2.	Number of swapped pairs: 18857 / 124750	nDCG: 0.9467
epoch: 2.	Number of swapped pairs: 17492 / 124750	nDCG: 0.9533
epoch: 2.	Number of swapped pairs: 16105 / 124750	nDCG: 0.9580
epoch: 2.	Number of swapped pairs: 14896 / 124750	nDCG: 0.9621
epoch: 2.	Number of swapped pairs: 13677 / 124750	nDCG: 0.9663
